# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG =  or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
 'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[1.986 1.983 2.001 1.988 2.039 2.004 1.993 2.047 2.055 2.046]
 [3.073 2.918 2.873 3.03  2.964 2.941 3.062 3.008 2.886 3.041]
 [3.97  3.936 3.984 4.065 3.977 4.005 4.006 4.006 4.111 3.904]] [[0.991804 0.980711 0.978999 1.031856 1.087479 1.031984 0.944951 1.038791
  1.093975 1.007884]
 [2.035671 1.895276 1.902871 1.9891   2.078704 1.883519 1.900156 2.107936
  1.851004 2.125319]
 [2.9631   2.927904 3.131744 3.046775 2.900471 2.938975 3.139964 2.867964
  2.886679 2.970784]]
Mean and variance endomwnets:
[[1.984 1.983 1.979 2.035 1.971 2.046 2.006 1.958 2.016 1.966]
 [3.064 2.944 2.959 2.988 2.965 2.997 3.008 3.053 2.975 2.949]
 [4.049 4.171 4.046 4.007 3.973 4.034 3.998 4.044 4.006 3.832]] [[1.037744 0.970711 0.972559 1.075775 0.948159 1.019884 0.953964 0.942236
  0.909744 0.942844]
 [2.325904 1.980864 1.981319 2.093856 1.999775 1.920991 2.045936 1.940191
  1.870375 1.876399]
 [2.916599 3.379759 2.969884 2.774951 2.794271 2.990844 3.059996 2.960064
  3.101964

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG), \
                      'Budget' : lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold
Lower Solutions:
[[0.      0.13022]
 [0.      0.13022]
 [0.15472 0.     ]]
Writing to file data.csv
Guardrail-0.5
Lower and Upper Solutions:
[[0.      0.13152]
 [0.      0.13152]
 [0.15565 0.     ]]
[[0.      0.19235]
 [0.      0.19235]
 [0.22763 0.     ]]
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[0.     0.1301]
 [0.     0.1301]
 [0.1533 0.    ]]
[[0.      0.26082]
 [0.      0.26082]
 [0.30733 0.     ]]
Out of Budget!
Out of Budget!
Writing to file data.csv
Guardrail-0.25
Lower and Upper Solutions:
[[0.      0.12937]
 [0.      0.12937]
 [0.15388 0.     ]]
[[0.      0.29559]
 [0.      0.29559]
 [0.35161 0.     ]]
Writing to file data.csv
traj len 1000
traj len 1000
traj len 1000
traj len 1000
        Algorithm     Reward      Time     Space  Efficiency  Hindsight Envy  \
0  FixedThreshold -10.825881  5.418436 -14837.94 -674.632020     -101.691000   
1   Guardrail-0.5  -7.785647  5.216183 -12666.22 -200.791771     -155.457934   
2   Guardrail-0.3 

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)